In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
root = '/content/drive/My Drive/red2'

In [4]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD

from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator

#con 1024 capas en la ultima convolucion
batch_size = 64
num_classes = 10
epochs = 250


#### LOAD AND TRANSFORM
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## DEFINE A DATA AUGMENTATION GENERATOR

datagen = ImageDataGenerator(
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)


## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGN(model,filters,ishape=0):
  if (ishape!=0):
    model.add(Conv2D(filters, (3, 3), padding='same',
                 input_shape=ishape))
  else:
    model.add(Conv2D(filters, (3, 3), padding='same'))
  
  model.add(BN())
  model.add(Activation('relu'))

  model.add(Conv2D(filters, (3, 3), padding='same'))
  model.add(BN())
  model.add(Activation('relu'))

  model.add(Conv2D(filters, (3, 3), padding='same'))
  model.add(BN())
  model.add(Activation('relu'))

  model.add(Conv2D(filters, (3, 3), padding='same'))
  model.add(BN())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  
  return model

  
## DEF NN TOPOLOGY  
model = Sequential()

model=CBGN(model,32,x_train.shape[1:])
model=CBGN(model,64)
model=CBGN(model,512)
model=CBGN(model,512)
model=CBGN(model,512)

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()


## OPTIM AND COMPILE
opt = SGD(lr=0.1, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
def scheduler(epoch):
    if epoch < 100:
        return .1
    elif epoch < 170:
        return 0.01
    else:
        return 0.001

set_lr = LRS(scheduler)

model_checkpoint_callback = ModelCheckpoint(
    filepath=root,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

## TRAINING with DA and LRA
history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[set_lr,model_checkpoint_callback],
                            verbose=1)


## TEST
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

(50000, 32, 32, 3)
(10000, 32, 32, 3)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/250
781/781 [==============================] - 66s 74ms/step - loss: 2.1483 - accuracy: 0.1993 - val_loss: 2.3409 - val_accuracy: 0.2541
Epoch 2/250
781/781 [==============================] - 58s 74ms/step - loss: 1.5456 - accuracy: 0.4221 - val_loss: 1.8413 - val_accuracy: 0.4036
Epoch 3/250
781/781 [==============================] - 58s 74ms/step - loss: 1.2471 - accuracy: 0.5511 - val_loss: 1.2923 - val_accuracy: 0.5419
Epoch 4/250
781/781 [==============================] - 58s 74ms/step - loss: 0.9864 - accuracy: 0.6578 - val_loss: 0.9135 - val_accuracy: 0.6870
Epoch 5/250
781/781 [==============================] - 58s 74ms/step - loss: 0.8125 - accuracy: 0.7251 - val_loss: 0.9098 - val_accuracy: 0.7102
Epoch 6/250
781/781 [==============================] - 58s 74ms/step - loss: 0.6954 - accuracy: 0.7611 - val_loss: 0.9663 - val_accuracy: 0.7062
Epoch 7/250
781/781 [==============================] - 58s 74ms/step - loss: 0.6192 - accuracy: 0.7897 - val_loss: 0.7027 - val_ac

In [5]:
model.load_weights(root)
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 5s 15ms/step - loss: 0.5155 - accuracy: 0.9203
Test loss: 0.51554274559021
Test accuracy: 0.9203000068664551
